# Rankings

In [ ]:
ENV="dev"

## Connection to PostgreSQL database

In [ ]:
from itables import init_notebook_mode, show

from postgres.PostgresToDataFrame import PostgresToDataframe
from src.rankings.TeamsRanking import TeamsRanking

init_notebook_mode(all_interactive=True)

In [ ]:
db = PostgresToDataframe(ENV)

## Rankings

In [ ]:
ID_CHP = 'ligue_1'
SEASON = '2023-2024'
SIDE = 'both'
FIRST_WEEK = 1
LAST_WEEK = 100

N = 1000000
ROUND = 2

### Teams ranking

In [ ]:
teams_ranking = TeamsRanking(db)
truc = teams_ranking.build_ranking(ID_CHP, SEASON, SIDE, FIRST_WEEK, LAST_WEEK, N, ROUND)

In [ ]:
show(truc.sort_values('xP', ascending=False), paging=False)

In [ ]:
show(truc[['Matches', 'Club', 'Wins', 'Draws', 'Loses', 'Points', 'xP', 'Diff Points']].set_index('Club').loc['FC Lorient'], paging=False)

### Players Rankings

In [ ]:
db.execute_sql_file("sql/rankings/players.sql")

In [ ]:
players_ranking = db.df_from_query("select * from players_rankings(id_chp := 'ligue_1', id_season := '2023-2024', side := 'both');").set_index("Ranking")

In [ ]:
players_ranking

## Distinct positions

In [ ]:
all_positions = db.df_from_query("select position from player_stats ps group by position;")["position"].dropna().tolist()
flattened_positions = []
for sublist in [position.split(',') for position in all_positions]:
    flattened_positions.extend(sublist)

In [ ]:
list(set(all_positions))

In [ ]:
unique_positions = list(set(flattened_positions))
general_positions = ['gk', 'def', 'mid', 'att']

{
    'gk': 'gk',
    'df': 'def',
    'lb': 'def',
    'cb': 'def',
    'wb': 'def', #wide back
    'rb': 'def',
    'dm': 'mid',
    'cm': 'mid',
    'lm': 'mid',
    'rm': 'mid',
    'mf': 'mid', #in-sub players
    'am': 'mid', #attacking midfielder
    'lw': 'att', #winger
    'rw': 'att',
    'fw': 'att'
}

# Closing the PostgreSQL session

In [ ]:
db.close()